In [1]:
import pickle
# import graphvite as gv
import dataset
import nltk
import re
import spacy
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')



%reload_ext autoreload
%autoreload

[nltk_data] Downloading package punkt to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /var/home/zaristei/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings

In [3]:
alias2entity = dataset.wikidata5m.load_alias("entity.txt.gz")

In [4]:
print(entity2id[alias2entity["invented"]])

2580138


In [5]:
nlp = spacy.load('en_core_web_sm') 
sentence = "Who invented machine learning, was it Steve Jobs"
doc = nlp(sentence) 
print(doc)
for ent in doc.ents: 
    print(ent.text, ent.label_) 

Who invented machine learning, was it Steve Jobs
Steve Jobs PERSON


In [6]:
def clean_text(text):
    # Extract named entities
    nlp = spacy.load('en_core_web_sm') 
    named_entities = nlp(text).ents

    # Tokenize the text
    words = word_tokenize(text)

    # Remove punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Part-of-speech tagging
    tagged_words = pos_tag(words)

    cleaned_text = ' '.join(words).lower()
    
    return cleaned_text, named_entities

# Example usage:
text = "Who invented machine learning, was it Steve Jobs?"
cleaned_text, named_entities = clean_text(text)
print("Cleaned Text:", cleaned_text)
print("Named Entities:", named_entities)

Cleaned Text: invented machine learning steve jobs
Named Entities: (Steve Jobs,)


In [7]:
def extract_continuous_word_sets(text, window_size):
    words = text.split()
    word_sets = []
    
    for i in range(len(words) - window_size + 1):
        word_set = ' '.join(words[i:i + window_size])
        word_sets.append(word_set)

    return word_sets

In [8]:
def query_to_kg_embeddings(query):
    wiki_embeddings = {}
    cleaned_text, named_entities = clean_text(query)
    for ne in named_entities: 
        try:
            ne = ne.orth_.lower()
            wiki_embeddings[ne] = entity_embeddings[entity2id[alias2entity[ne]]]
            cleaned_text = cleaned_text.replace(ne, "")
        except Exception as e:
            print("KeyError: ", e)
    num_of_words = len(word_tokenize(cleaned_text))
    for i in range(num_of_words, 0, -1):
        word_sets = extract_continuous_word_sets(cleaned_text, i)
        for word in word_sets:
            try:
                wiki_embeddings[word] = entity_embeddings[entity2id[alias2entity[word]]]
                cleaned_text = cleaned_text.replace(word, "")
            except Exception as e:
                print("KeyError: ", e)

    return wiki_embeddings

In [9]:
def query_to_node_id(query):
    node_ids = {}
    cleaned_text, named_entities = clean_text(query)
    for ne in named_entities: 
        try:
            ne = ne.orth_.lower()
            node_ids[ne] = alias2entity[ne]
            cleaned_text = cleaned_text.replace(ne, "")
        except Exception as e:
            pass
            #print("KeyError: ", e)
    num_of_words = len(word_tokenize(cleaned_text))
    for i in range(num_of_words, 0, -1):
        word_sets = extract_continuous_word_sets(cleaned_text, i)
        for word in word_sets:
            try:
                node_ids[word] = alias2entity[word]
                cleaned_text = cleaned_text.replace(word, "")
            except Exception as e:
                pass
                #print("KeyError: ", e)
    return node_ids    

In [10]:
node_ids = (query_to_node_id("Who invented machine learning, was it Isaac Newton?"))
node_ids

{'isaac newton': 'Q935', 'machine learning': 'Q2539', 'invented': 'Q18119757'}

In [11]:
import os
halueval_path = os.path.join(os.getcwd(), "submodules", "HaluEval", "data", "qa_data.json")
print(halueval_path)

/var/home/zaristei/repos/KG-LLM-Hallucination/submodules/HaluEval/data/qa_data.json


In [12]:
import json
with(open(halueval_path, "r")) as f:
    data_lst = f.readlines()

data_lst = [json.loads(data) for data in data_lst]

In [13]:
RELEVANT_KEYS = ['knowledge', 'question']

In [14]:
import tqdm

In [15]:
def process_data(data):
    result = {}
    for k in RELEVANT_KEYS:
        node_vals = query_to_node_id(data[k])
        result.update(node_vals)
        
    return result

In [16]:
processed_data = [process_data(i) for i in data_lst[:10]]

In [17]:
len(processed_data)

10

In [18]:
from tqdm.contrib.concurrent import process_map
import multiprocessing as mp
import time

In [19]:
mp.cpu_count()

24

In [20]:
start = time.time()
result = process_map(process_data, data_lst, max_workers=5)
'''
result = []
for data in tqdm.tqdm(data_lst):
    result.append(process_data(data))
'''
end = time.time()
print(end-start)

/tmp/ipykernel_15925/3956094896.py:2: TqdmWarning: Iterable length 10000 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  result = process_map(process_data, data_lst, max_workers=5)


  0%|          | 0/10000 [00:00<?, ?it/s]

949.5897569656372


In [21]:
from tqdm.contrib.concurrent import thread_map
start = time.time()
if __name__ == "__main__":
    result = process_map(process_data, data_lst)
#result = []
#for data in tqdm.tqdm(data_lst):
#    result.append(process_data(data))
end = time.time()
print(end-start)

/tmp/ipykernel_15925/2795308879.py:4: TqdmWarning: Iterable length 10000 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  result = process_map(process_data, data_lst)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
pickle.dump(result, open("processed_node_ids.pkl", "wb"))